# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2024-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>

# <h1><center>Actividad 04: Obteniendo Datos de la Web</center></h1>

Esta actividad busca aplicar los conocimientos en lectura de datos de la web en diferentes formas y conectar la información.

## Instrucciones

Esto Notebook contiene las instrucciones a realizar para la actividad. 

<b>Al finalizarla, deben subir el Notebook y los archivos generados en un único archivo .zip, al módulo de la Actividad 04 en Canvas. Entregas posteriores al cierre de la actividad serán evaluadas con nota 1.0.</b>

## Actividad

En la Clase 4 analizamos los datos de la Filarmónica de NY y, si revisaron la solución que publicamos posteriormente, pudimos ver que el compositor con más obras interpretadas por la Filarmónica fue Richard Wagner. Otro de los compositores que apareció en la lista es Wolfgang Amadeus Mozart.

Wolfgang Amadeus Mozart fue un compositor muy prolífico que vivió ente 1756 y 1791. Entre sus obras, hay una cantidad importante de composiciones hechas para obras de teatro, incluyendo óperas, algunas muy famosas. En esta actividad queremos saber cuántas veces han sido interpretadas por la Filarmónica, las obras de ópera de Mozart, y cuál es la que más ha sido interpretada.

No tenemos una única fuente de datos en este caso, así que tendremos que conectar diferentes fuentes para lograr contestar esta pregunta. Para ello haga lo siguiente:
<ol>

<li>Obras para escenario</li>

Lo primero que debemos hacer es buscar cuáles son todas las óperas que hizo Mozart en su vida. Por suerte, hay una página de Wikipedia que contiene una ordenada tabla con toda la información necesaria. Esta la pueden encontrar en:

https://en.wikipedia.org/wiki/List_of_operas_by_Wolfgang_Amadeus_Mozart

Usando la librería `requests` y `BeautifulSoup` baje la información de la tabla con el listado de los trabajos de Mozart para ópera, creando un DataFrame con toda la información sólo de las columnas 'Period', 'Title', 'Genre' y 'Köchel No.'.

<br>
    
<li>Descargar datos de la Filarmónica</li>

Aprendimos en la Clase 04 que la Filarmónica de NY tiene un repositorio de GitHub con los datos de todas las obras interpretadas desde 1842. El archivo JSON con toda la que se actualiza semana a semana está disponible en

https://raw.githubusercontent.com/nyphilarchive/PerformanceHistory/main/Programs/json/complete.json
    
Usando la librería `requests` descargue y guarde en un archivo en su repositorio local.

<br>
    
<li>Procesar los datos de la Filarmónica</li>

Usando los elementos que están bajo la categoría `programs`, construya un DataFrame con los datos bajados del repositorio de la Filarmónica. Aplánelos usando la categoría `works` y genere un DataFrame filtrado que sólo tenga las obras de Mozart tocadas por la Filarmónica.
    
<br>

<li>Conectar las bases de datos</li>
<br>

Conecte ambos DataFrames para generar un único DataFrame final, que tenga todas las columnas solicitadas de Wikipedia, y una columna adicional, llamada `ny phil` que contenga la cantidad de veces que la Orquesta Filarmónica ha interpretado esa obra. En esa sección podrá encontrar algunos códigos de apoyo para realizar esa parte.
    
<br>
    
<li>Pregunta</li>

Usando la información de la nueva base de datos generada conteste, ¿Cuál es la 'opera de Mozart que más veces ha sido interpretada por la Orquesta Filarmónica de NY?

## Rúbrica

- Si han logrado todo: 7.0
- Si han logrado hasta el punto 3: 5.0
- Otro caso: 1.0

In [1]:
# librerías necesarias para el ejercicio
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json

## 1. Obras para escenario

A continuación use las celdas para leer y transformar en un DataFrame los datos de la tabla de óperas de Mozart. Como recomendación, revise la primera tabla que aparece, pues es la que tiene la información relevante. 

**IMPORTANTE**: La columna Köchel No, indica el ID de la obra en el cátalogo creado por Ludwig Ritter von Köchel. Esta es muy importante porque es como el RUT de la obra y nos permite revisar otras bases de datos y comprar, sin necesidad de traducir los títulos que a veces pueden venir con errores o cambios por diferentes idiomas. Le recomendamos limpiar esa columna de tal forma que sólo quede el número Köchel (formato K.xxx) para poder usarlo después.

In [2]:
# leemos primero la información
url = "https://en.wikipedia.org/wiki/List_of_operas_by_Wolfgang_Amadeus_Mozart"
page = requests.get(url).text

In [3]:
# usando beautifoulsoup la transformamos para poder hacer análisis sobre los datos.
soup = bs(page)

In [4]:
# cargamos todas las tablas dispobibles
tables = soup.find_all('table')
len(tables)

4

In [5]:
# y tomamos la primera
table = tables[0]
#table

In [6]:
#creamos un DataFrame vacío con los títulos de la tabla
df = pd.DataFrame(columns = ['periodo','titulo','genero', 'kochel'])

# iterar sobre cada fila ('tr') para completar la información
for row in table.find_all('tr')[1::]:
    cols = row.find_all("td")
    #print(cols)
    if len(cols) > 7:
        cols = [col.text.strip() for col in cols]
        #print(cols)
        periodo = cols[0]
        titulo = cols[1]
        genero = cols[2]
        kochel = cols[8]
        # procesar el kochel
        if 'S' in kochel:
            idx = kochel.find('S')
        if 'L' in kochel:
            idx = min(kochel.find('L'), idx)
        if '(' in kochel:
            idx = min(kochel.find('('), idx)
        kochel = kochel[0:idx]
        df = df.append({'periodo': periodo, 'titulo': titulo, 'genero': genero, 'kochel': kochel}, ignore_index=True)
df

C:\Users\rodri\AppData\Local\Temp\ipykernel_12872\1906119242.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'periodo': periodo, 'titulo': titulo, 'genero': genero, 'kochel': kochel}, ignore_index=True)
C:\Users\rodri\AppData\Local\Temp\ipykernel_12872\1906119242.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'periodo': periodo, 'titulo': titulo, 'genero': genero, 'kochel': kochel}, ignore_index=True)
C:\Users\rodri\AppData\Local\Temp\ipykernel_12872\1906119242.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'periodo': periodo, 'titulo': titulo, 'genero': genero, 'kochel': kochel}, ignore_index=True)
C:\Users\rodri\AppData\Local\Temp\ipykernel_12872\1906119242.

,periodo,titulo,genero,kochel
0,1766–67,"Die Schuldigkeit des ersten Gebots, Part 1[n](...",Sacred Singspiel(collaboration),K.35
1,1767,Apollo et Hyacinthus(Apollo and Hyacinth),Music for a Latin drama[18],K.38
2,1768,Bastien und Bastienne(Bastien and Bastienne),Singspiel1 act,K.50/46b
3,1768,La finta semplice(The feigned simpleton),Opera buffa3 acts,K.51/46a
4,1770,"Mitridate, re di Ponto(Mithridates, King of Po...",Opera seria3 acts,K.87/74a
5,1771,Ascanio in Alba(Ascanius in Alba),Festspiel[t]2 acts,K.111
6,1772,Il sogno di Scipione(Scipio's Dream),"Azione teatrale, or Serenata drammatica1 act",K.126
7,1772,Lucio Silla,Dramma per musica3 acts,K.135
8,1774–75,La finta giardiniera(The pretend garden-maid),Dramma giocoso3 acts[x],K.196
9,1775,Il re pastore(The Shepherd King),Serenata2 acts,K.208


## 2. Descargar datos de la Filarmónica

A continuación use las celdas para descargar los datos actualizados de la Filarmónica de NY.

In [7]:
# bajamos los datos y los escribimos en forma local.
url = 'https://raw.githubusercontent.com/nyphilarchive/PerformanceHistory/main/Programs/json/complete.json'
respuesta = requests.get(url)
open('nyphilharmonic.json','wb').write(respuesta.content)

37231137

## 3. Procesar los datos de la Filarmónica

A continuación procese los datos del archivo JSON de la filarmónica para crear un DataFrame con todas las interpretaciones de obras de Mozart. Revise la Clase 04 para ver cómo hacer parte de este proceso, y use el filtro `'Mozart,  Wolfgang  Amadeus'` para separar sus obras.

In [8]:
# leemos los datos
datos_raw = json.load(open('nyphilharmonic.json','r', encoding='UTF-8'))

In [9]:
# vemos los datos por programa
nycphil = pd.json_normalize(datos_raw['programs'])
nycphil.head()

,id,programID,orchestra,season,concerts,works
0,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,"[{'eventType': 'Subscription Season', 'Locatio...","[{'ID': '52446*', 'composerName': 'Beethoven, ..."
1,1118e84e-eb59-46cc-9119-d903375e65e6-0.1,5178,New York Philharmonic,1842-43,"[{'eventType': 'Subscription Season', 'Locatio...","[{'ID': '52437*', 'composerName': 'Beethoven, ..."
2,08536612-27c3-437e-9b44-def21034b06c-0.1,10785,Musicians from the New York Philharmonic,1842-43,"[{'eventType': 'Special', 'Location': 'Manhatt...","[{'ID': '52364*1', 'composerName': 'Beethoven,..."
3,81a3b8de-1737-4c9e-9318-b839f7c7c4c0-0.1,5887,New York Philharmonic,1842-43,"[{'eventType': 'Subscription Season', 'Locatio...","[{'ID': '52434*', 'composerName': 'Beethoven, ..."
4,09581bb7-8855-4965-b302-fc54cc669041-0.1,305,New York Philharmonic,1843-44,"[{'eventType': 'Subscription Season', 'Locatio...","[{'ID': '52453*', 'composerName': 'Beethoven, ..."


In [10]:
# aplanamos el DataFrame por compositor y luego filtramos los de Mozart
table = pd.json_normalize(datos_raw["programs"], record_path="works")
wam = table[table['composerName'] == 'Mozart,  Wolfgang  Amadeus']
wam.head(10)

,ID,composerName,workTitle,conductorName,soloists,movement,interval,movement._,movement.em,workTitle._,workTitle.em
7,8336*4,"Mozart, Wolfgang Amadeus","ABDUCTION FROM THE SERAGLIO,THE, K.384","Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...","""Ach Ich liebte,"" Konstanze (aria)",NaN,NaN,NaN,NaN,NaN
27,8955*13,"Mozart, Wolfgang Amadeus","MAGIC FLUTE, THE, K.620",Not conducted,"[{'soloistName': '', 'soloistInstrument': '', ...",Aria (unspecified),NaN,NaN,NaN,NaN,NaN
40,8955*1,"Mozart, Wolfgang Amadeus","MAGIC FLUTE, THE, K.620","Hill, Ureli Corelli",[],Overture,NaN,NaN,NaN,NaN,NaN
43,51668*,"Mozart, Wolfgang Amadeus","SYMPHONY NO. 41, C MAJOR, K.551, ""JUPITER""","Etienne, Denis G.",[],NaN,NaN,NaN,NaN,NaN,NaN
77,8863*4,"Mozart, Wolfgang Amadeus","CLEMENZA DI TITO, LA, K.621","Hill, Ureli Corelli","[{'soloistName': 'Loder, Edward', 'soloistInst...","""Deh, per questo istante solo""",NaN,NaN,NaN,NaN,NaN
115,51668*2,"Mozart, Wolfgang Amadeus","SYMPHONY NO. 41, C MAJOR, K.551, ""JUPITER""","Boucher, Alfred",[],Andante cantabile,NaN,NaN,NaN,NaN,NaN
118,51664*,"Mozart, Wolfgang Amadeus","SYMPHONY NO. 40, G MINOR, K.550","Timm, Henry C.",[],NaN,NaN,NaN,NaN,NaN,NaN
126,8955*1,"Mozart, Wolfgang Amadeus","MAGIC FLUTE, THE, K.620","Hill, Ureli Corelli",[],Overture,NaN,NaN,NaN,NaN,NaN
138,51658*,"Mozart, Wolfgang Amadeus","SYMPHONY NO. 39, E-FLAT MAJOR, K.543","Timm, Henry C.",[],NaN,NaN,NaN,NaN,NaN,NaN
149,8849*5,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527","Boucher, Alfred","[{'soloistName': 'Loder, Edward', 'soloistInst...","""Batti, batti,"" Zerlina, Act I, scene xvi",NaN,NaN,NaN,NaN,NaN


## 4. Conectar las bases de datos

En la columna `workTitle` del DataFrame de Mozart, debería tener el nombre completo de cada una de las obras interpretadas. Lamentablemente, este nombre no necesariamente coincide en forma exacta con el nombre de la tabla de Wikipedia. Por suerte, tanto la tabla de Wikipedia como la columna `workTitle` tienen un elemento identificador único: el número Köchel. Como comentamos antes, úmero Köchel es un índice musicológico de las obras de Mozart y es único por obra. La tabla de Wikipedia la tiene como una columna, pero la de la Filarmónica la tiene al final del texto del nombre de cada obra, por lo que tendremos que trabajar para extraerla.

Por ejemplo, si queremos encontrar el texto `K.527` que corresponde a la famosa ópera "Don Giovanni", una forma fácil de hacerlo es la siguiente. Suponga que `wam` es el nombre del DataFrame con todas las obras de Mozart interpretadas por la filarmónica. Entonces podemos filtrar las filas que tienen ese número Köchel con el siguiente código:

`textToSearch = 'K.527'`

`wam[wam['workTitle'].str.contains(textToSearch)]`


Pruebe esto en la siguiente celda.

In [11]:
# ejemplo de búsqueda
textToSearch = 'K.527'
wam[wam['workTitle'].str.contains(textToSearch)]

,ID,composerName,workTitle,conductorName,soloists,movement,interval,movement._,movement.em,workTitle._,workTitle.em
149,8849*5,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527","Boucher, Alfred","[{'soloistName': 'Loder, Edward', 'soloistInst...","""Batti, batti,"" Zerlina, Act I, scene xvi",NaN,NaN,NaN,NaN,NaN
188,8849*10,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527","Loder, George","[{'soloistName': 'Arthurson', 'soloistInstrume...","""Il mio tesoro intanto,"" Don Ottavio, Act II, ...",NaN,NaN,NaN,NaN,NaN
276,8849*14,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527","Eisfeld, Theodore","[{'soloistName': 'Bouchelle, Wallace', 'solois...","""In quali eccessi, o numi...Mi tradi,"" Donna E...",NaN,NaN,NaN,NaN,NaN
349,8849*4,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527","Eisfeld, Theodore","[{'soloistName': 'Bostwick, Emma Gillingham', ...","""Crudele! Ah! mio bene...Non mi dir,"" Donna An...",NaN,NaN,NaN,NaN,NaN
456,8849*4,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527","Eisfeld, Theodore","[{'soloistName': 'La Grange, Anna De', 'solois...","""Crudele! Ah! mio bene...Non mi dir,"" Donna An...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
79331,8849*28,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527","Gilbert, Alan; Tritle, Kent; Weilerstein, Jo...","[{'soloistName': 'McKinney, Ryan', 'soloistIns...",Act I Finale,NaN,NaN,NaN,NaN,NaN
82370,8849*8,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527","Lewis, Courtney","[{'soloistName': 'Salters, Stephen', 'soloistI...","""Fin ch'han dal vino""",NaN,NaN,NaN,NaN,NaN
83247,9176*2,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527: SUITE FOR WINDS (ARR. Tr...",NaN,"[{'soloistName': 'Sylar, Sherry', 'soloistInst...","Madamina, il catalogo è questo",NaN,NaN,NaN,NaN,NaN
83248,9176*3,"Mozart, Wolfgang Amadeus","DON GIOVANNI, K.527: SUITE FOR WINDS (ARR. Tr...",NaN,"[{'soloistName': 'Sylar, Sherry', 'soloistInst...",Là ci darem la mano,NaN,NaN,NaN,NaN,NaN


Ahora agregue una nueva columna al DataFrame de Wikipedia, llamado `ny_phil`, agregando la cantidad de veces que la Filarmónica tocó cada obra. Para ello agregue una columna vacía de la siguiente forma:
`
df['ny phil'] = None
`

y luego itere sobre cada fila del DataFrame agregando la cantidad de veces que ocurrió cada obra. Use el comando `len()` para contar la cantidad de veces de cada obra. Puede iterar sobre el DataFrame usando el atributo `df.index`

**IMPORTANTE**: con el método de filtrado anterior, la búsqueda de la obra `K.35` va a arrojar también las obras `K.350` o `K.351` dado que contienen `K.35` como parte del string. Si logra limpiar eso para las obras `K.35` y `K.38` tendrá +1.0 de bono en esta actividad, pero si no lo logra hacer, no será penalizado por ello.

In [12]:
df['ny phil'] = None

for idx in df.index:
    kochel = df['kochel'][idx]
    if (kochel == 'K.35') or (kochel == 'K.38'):
        textToSearch = kochel + " "
    else:
        textToSearch = kochel
    count = len(wam[wam['workTitle'].str.contains(textToSearch)])
    
    df['ny phil'][idx] = count

In [13]:
df

,periodo,titulo,genero,kochel,ny phil
0,1766–67,"Die Schuldigkeit des ersten Gebots, Part 1[n](...",Sacred Singspiel(collaboration),K.35,0
1,1767,Apollo et Hyacinthus(Apollo and Hyacinth),Music for a Latin drama[18],K.38,0
2,1768,Bastien und Bastienne(Bastien and Bastienne),Singspiel1 act,K.50/46b,0
3,1768,La finta semplice(The feigned simpleton),Opera buffa3 acts,K.51/46a,0
4,1770,"Mitridate, re di Ponto(Mithridates, King of Po...",Opera seria3 acts,K.87/74a,0
5,1771,Ascanio in Alba(Ascanius in Alba),Festspiel[t]2 acts,K.111,0
6,1772,Il sogno di Scipione(Scipio's Dream),"Azione teatrale, or Serenata drammatica1 act",K.126,0
7,1772,Lucio Silla,Dramma per musica3 acts,K.135,2
8,1774–75,La finta giardiniera(The pretend garden-maid),Dramma giocoso3 acts[x],K.196,1
9,1775,Il re pastore(The Shepherd King),Serenata2 acts,K.208,17


## 5. Contestar la pregunta

Ahora podemos contestar la pregunta original, ¿cuál es la ópera de Mozart más tocada por la Filarmónica de NY?

Escriba el nombre directamente en la siguiente celda (no necesita automatizar el proceso usando código).

Vemos que la obra con mayor número de interpretaciones es entonces `Le nozze di Figaro (The Marriage of Figaro)`.